In [158]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

W1 = np.array([[1, 1, 1, 1, 1]]).T
b1 = np.array([[0, -1, -2, -3, -4]]).T
W2 = np.array([[3.2, -3, 1.4, 2.2, -1]])
b2 = np.array([[-0.2]])
print("Shapes:", W1.shape, b1.shape, W2.shape, b2.shape)

m = gp.Model("GNN Inverse")

x = m.addMVar((1, 1), lb=-float("inf"), ub=float("inf"), name="x")

activation_list = [x]
for i, layer, bias in [(1, W1, b1), (2, W2, b2)]:
    # print(bias)
    ts = m.addMVar((layer.shape[0], 1), lb=0, ub=100, name=f"tj{i}")
    ss = m.addMVar((layer.shape[0], 1), lb=0, ub=100, name=f"sj{i}")
    zs = m.addMVar((layer.shape[0], 1), vtype=GRB.BINARY, name=f"zj{i}")

    # m.addConstr(ts-ss==gp.quicksum(layer @ activation_list[-1] + bias)) # wrong, but something like this
    for j in range(layer.shape[0]):
        c = m.addConstr(ts[j]-ss[j]==gp.quicksum(layer[j, :] @ activation_list[-1] + bias[j]))

    m.addConstr(ts <= 10000*zs)
    m.addConstr(ss <= 10000*(1-zs))

    activation_list.append(ts)
# m.setObjective((ts[0][0]-1.5)*(ts[0][0]-1.5), GRB.MINIMIZE)
m.setObjective((ts-1.5)*(ts-1.5), GRB.MINIMIZE)

m.update()
m.display()


Shapes: (5, 1) (5, 1) (1, 5) (1, 1)
Minimize
  2.25 + -3.0 tj2[0,0] + [ tj2[0,0] ^ 2 ]
Subject To
  R0: -1.0 x[0,0] + tj1[0,0] + -1.0 sj1[0,0] = 0
  R1: -1.0 x[0,0] + tj1[1,0] + -1.0 sj1[1,0] = -1
  R2: -1.0 x[0,0] + tj1[2,0] + -1.0 sj1[2,0] = -2
  R3: -1.0 x[0,0] + tj1[3,0] + -1.0 sj1[3,0] = -3
  R4: -1.0 x[0,0] + tj1[4,0] + -1.0 sj1[4,0] = -4
  R5: tj1[0,0] + -10000.0 zj1[0,0] <= -0
  R6: tj1[1,0] + -10000.0 zj1[1,0] <= -0
  R7: tj1[2,0] + -10000.0 zj1[2,0] <= -0
  R8: tj1[3,0] + -10000.0 zj1[3,0] <= -0
  R9: tj1[4,0] + -10000.0 zj1[4,0] <= -0
  R10: sj1[0,0] + 10000.0 zj1[0,0] <= 10000
  R11: sj1[1,0] + 10000.0 zj1[1,0] <= 10000
  R12: sj1[2,0] + 10000.0 zj1[2,0] <= 10000
  R13: sj1[3,0] + 10000.0 zj1[3,0] <= 10000
  R14: sj1[4,0] + 10000.0 zj1[4,0] <= 10000
R15: -3.2 tj1[0,0] + 3.0 tj1[1,0] + -1.4 tj1[2,0] + -2.2 tj1[3,0] + tj1[4,0] + tj2[0,0]
 + -1.0 sj2[0,0] = -0.2
  R16: tj2[0,0] + -10000.0 zj2[0,0] <= -0
  R17: sj2[0,0] + 10000.0 zj2[0,0] <= 10000
Bounds
  x[0,0] free
  0 <= tj

In [159]:
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Xeon(R) Gold 5120 CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 14 physical cores, 28 logical processors, using up to 28 threads

Optimize a model with 18 rows, 19 columns and 46 nonzeros
Model fingerprint: 0x9280d1ee
Model has 1 quadratic objective term
Variable types: 13 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [3e+00, 3e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+02]
  RHS range        [2e-01, 1e+04]
Presolve time: 0.00s
Presolved: 18 rows, 19 columns, 46 nonzeros
Presolved model has 1 quadratic objective terms
Variable types: 13 continuous, 6 integer (6 binary)
Found heuristic solution: objective 2.2500000

Root relaxation: objective 0.000000e+00, 17 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Dep

In [160]:
# m.computeIIS()
# m.write("model.ilp")

for v in m.getVars():
    print('%s %g' % (v.VarName, v.X))

print('Obj: %g' % m.ObjVal)

x[0,0] 0.53125
tj1[0,0] 0.53125
tj1[1,0] 0
tj1[2,0] 0
tj1[3,0] 0
tj1[4,0] 0
sj1[0,0] 0
sj1[1,0] 0.46875
sj1[2,0] 1.46875
sj1[3,0] 2.46875
sj1[4,0] 3.46875
zj1[0,0] 1
zj1[1,0] -0
zj1[2,0] -0
zj1[3,0] -0
zj1[4,0] -0
tj2[0,0] 1.5
sj2[0,0] 0
zj2[0,0] 1
Obj: 0
